# Final Project

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
fname = "business.csv"
business_df = pd.read_csv(fname)

In [35]:
business_df = business_df[['doing_bus_as_name', 'address_str', 'zip','naics_description','lat','lon']]
business_df = business_df.loc[(business_df['naics_description'] == 'full-service restaurants') |
               (business_df['naics_description'] == 'cafeterias') | 
               (business_df['naics_description'] == 'food services & drinking places') |
               (business_df['naics_description'] == 'limited-service eating places') |
               (business_df['naics_description'] == 'limited-service restaurants')  |
               (business_df['naics_description'] == 'mobile food services') |
               (business_df['naics_description'] == 'drinking places (alcoholic beverages)') |
               (business_df['naics_description'] == 'snack & nonalcoholic beverage bars')]
business_df

,doing_bus_as_name,address_str,zip,naics_description,lat,lon
11,c r e a m,4808 talmadge park row,92115-1939,mobile food services,32.767243,-117.096294
14,snow cones y raspados,740 s 46th st 1/2,92113-2911,mobile food services,32.697974,-117.096250
17,jalapeno taco shop,3130 university ave,92104-2047,full-service restaurants,32.748683,-117.126968
39,tacos el campechano inc,623 national city blvd,91950-1121,mobile food services,32.697974,-117.096250
47,up2you cafe llc,7510 mesa college dr,92111-5000,limited-service eating places,32.718370,-117.157817
55,awash ethiopian restaurant,2104 el cajon blvd,92104-1102,full-service restaurants,32.677079,-117.107167
77,gueros taco shop,2709 imperial ave,92102-4019,limited-service restaurants,32.890137,-117.150877
79,the fire spot,4646 convoy st,92111-2315,full-service restaurants,32.773069,-117.156144
88,up2you cafe llc,7770 vickers st,92111-1545,food services & drinking places,33.016118,-117.075608
103,cold beers & cheeseburgers,322 05th ave,92101-6910,full-service restaurants,32.708791,-117.160357


NameError: name 'business_df' is not defined